In [1]:
import numpy as np


In [2]:
setup_time = np.array([
    [np.inf, 15.05, 12.37, 14.78, 15.23, np.inf],
    [13.70,  np.inf, 15.67, 13.12, 15.46, np.inf],
    [13.51, 13.91,  np.inf, 11.60, 11.86, np.inf],
    [16.49, 16.93, 13.55,  np.inf, 11.88, np.inf],
    [17.00, 16.56, 17.03, 17.28,  np.inf,np.inf],
    [10.58, 16.04, 13.82, 11.54, 14.88, np.inf] 
], dtype=float)

In [3]:

def tsp_nearest_neighbor(distance_matrix, start):
    n = len(distance_matrix)
    visited = [False] * n
    tour = [start]
    visited[start] = True
    total_distance = 0

    current_city = start
    for _ in range(n - 1):
        nearest_city = None
        nearest_distance = float("inf")
        for city in range(n):
            if not visited[city] and distance_matrix[current_city][city] < nearest_distance:
                nearest_city = city
                nearest_distance = distance_matrix[current_city][city]
        
        if nearest_city is None:  # safeguard
            break

        tour.append(nearest_city)
        visited[nearest_city] = True
        total_distance += nearest_distance
        current_city = nearest_city

    return tour, total_distance


tour, distance = tsp_nearest_neighbor(setup_time, start=5)
print("Nearest Neighbor Tour:", tour)
print("Total Distance:", distance)


Nearest Neighbor Tour: [5, 0, 2, 3, 4, 1]
Total Distance: 62.989999999999995


In [4]:
def greedy_tsp_asymmetric(distance_matrix):
    n = len(distance_matrix)
    edges = []

    # Step 1: collect valid directed edges
    for i in range(n):
        for j in range(n):
            if i != j and distance_matrix[i][j] != float("inf"):
                edges.append((distance_matrix[i][j], i, j))
    
    # Step 2: sort edges by weight
    edges.sort(key=lambda x: x[0])

    out_degree = [0] * n  # how many edges go out from each node
    in_degree = [0] * n   # how many edges go into each node
    tour_edges = []

    # Step 3: select edges
    for w, u, v in edges:
        if out_degree[u] < 1 and in_degree[v] < 1:
            # Avoid forming cycle before including all nodes
            if len(tour_edges) < n - 1 or (v == tour_edges[0][0] and u not in [x[0] for x in tour_edges]):
                tour_edges.append((u, v))
                out_degree[u] += 1
                in_degree[v] += 1
        if len(tour_edges) == n:
            break

    # Step 4: reconstruct tour from directed edges
    tour = [tour_edges[0][0]]
    current = tour_edges[0][0]
    while len(tour) < n:
        for u, v in tour_edges:
            if u == current:
                tour.append(v)
                current = v
                break
    # Step 5: compute distance
    total_distance = 0
    for i in range(len(tour)-1):
        total_distance += distance_matrix[tour[i]][tour[i+1]]

    return tour, total_distance


In [5]:
greedy_tsp_asymmetric(setup_time)

([5, 0, 2, 3, 4, 1], np.float64(62.989999999999995))

In [6]:
setup_atime = np.array([
    [np.inf, 15.05, 12.37, 14.78, 15.23],
    [13.70,  np.inf, 15.67, 13.12, 15.46],
    [13.51, 13.91,  np.inf, 11.60, 11.86 ],
    [16.49, 16.93, 13.55,  np.inf, 11.88],
    [17.00, 16.56, 17.03, 17.28,  np.inf],
    [10.58, 16.04, 13.82, 11.54, 14.88] 
], dtype=float)


In [7]:
from python_tsp.exact import solve_tsp_dynamic_programming

permutation, distance = solve_tsp_dynamic_programming(setup_time)
start_node = 5
if start_node in permutation:
    idx = permutation.index(start_node)
    permutation = permutation[idx:] + permutation[:idx]

# Recalculate distance for this rotated tour
new_distance = 0
for i in range(len(permutation)):
    u = permutation[i]
    v = permutation[(i + 1) % len(permutation)]  # wrap around to make it a cycle
    new_distance += setup_time[u, v]

print("Tour starting at node 5:", permutation)
print("Recalculated distance:", new_distance)



Tour starting at node 5: [5, 0, 1, 2, 3, 4]
Recalculated distance: inf


In [8]:
permutation

[5, 0, 1, 2, 3, 4]

In [9]:
distance

np.float64(inf)

In [10]:

import numpy as np
from scipy.optimize import linear_sum_assignment

INF = 1e12  # large cost to forbid arcs

def solve_atsp_path_assignment(setup_time, start=5, verbose=True, max_iters=1000):
    """
    Solve asymmetric path-TSP as assignment + subtour elimination.
    - setup_time: square numpy array of shape (n,n) with np.inf for forbidden/self arcs
    - start: fixed start node index
    Returns: (path, total_cost)
    """
    # --- shape and sanity checks ---
    assert setup_time.ndim == 2, "setup_time must be a 2D array"
    n, m = setup_time.shape
    assert n == m, "setup_time must be square"
    assert 0 <= start < n, "start must be a valid node index"

    # Build working cost matrix with large finite costs in place of inf
    C = setup_time.copy().astype(float)
    C[np.isinf(C)] = INF

    # Forbid inbound to start to enforce path starting at 'start'
    C[:, start] = INF

    # Forbid self loops
    for i in range(n):
        C[i, i] = INF

    # Maintain a mask of forbidden arcs to add cuts
    forbidden = np.zeros_like(C, dtype=bool)
    forbidden[:, start] = True
    np.fill_diagonal(forbidden, True)

    def run_hungarian(cost):
        row_ind, col_ind = linear_sum_assignment(cost)
        return row_ind, col_ind

    def extract_assignment_pairs(row_ind, col_ind):
        # returns dict next[i] = j (a permutation)
        nxt = {int(i): int(j) for i, j in zip(row_ind, col_ind)}
        return nxt

    def detect_cycles(nxt):
        # Return list of cycles, each as a list of nodes in order
        visited = {i: False for i in nxt.keys()}
        cycles = []
        for i in nxt.keys():
            if not visited[i]:
                cur = i
                cycle = []
                while not visited[cur]:
                    visited[cur] = True
                    cycle.append(cur)
                    cur = nxt[cur]
                # A cycle occurs if we came back to the start of this walk
                if cycle and nxt[cycle[-1]] == cycle:
                    cycles.append(cycle)
        return cycles

    def reachable_from_start(nxt, start):
        # Traverse following nxt starting at 'start'
        seen = set()
        cur = start
        while cur not in seen:
            seen.add(cur)
            cur = nxt[cur]
        return seen

    def is_single_path_from_start(nxt, start):
        # The assignment is a permutation (each outdeg=1, indeg=1).
        # For a Hamiltonian path starting at start (with inbound to start forbidden),
        # starting from 'start' must visit all nodes before any repeat.
        seen = reachable_from_start(nxt, start)
        return len(seen) == len(nxt)

    def path_from_nxt(nxt, start):
        order = [start]
        cur = start
        for _ in range(len(nxt)-1):
            cur = nxt[cur]
            order.append(cur)
        return order

    C_work = C.copy()
    it = 0
    while it < max_iters:
        it += 1
        # Apply current forbids
        C_work[forbidden] = INF

        # Run Hungarian
        row_ind, col_ind = run_hungarian(C_work)
        nxt = extract_assignment_pairs(row_ind, col_ind)

        if verbose:
            assign_cost = float(C[np.arange(n), col_ind].sum())
            print(f"Iteration {it}: assignment cost = {assign_cost:.6f}")
            print("Assignment pairs:", list(zip(row_ind.tolist(), col_ind.tolist())))

        # Check for a single Hamiltonian path from start
        if is_single_path_from_start(nxt, start):
            path = path_from_nxt(nxt, start)
            total_cost = 0.0
            for i in range(len(path)-1):
                total_cost += float(setup_time[path[i], path[i+1]])
            return path, total_cost

        # Otherwise detect cycles and cut one arc from a subtour not containing start
        cycles = detect_cycles(nxt)

        cut_done = False
        # Prefer cutting a subtour that does not include start
        for cyc in cycles:
            if start not in cyc:
                # Cut the cheapest selected arc in this subtour
                best_arc = None
                best_cost = +np.inf
                for u in cyc:
                    v = nxt[u]
                    cost_uv = C[u, v]
                    if cost_uv < best_cost:
                        best_cost = cost_uv
                        best_arc = (u, v)
                if best_arc is not None:
                    u, v = best_arc
                    forbidden[u, v] = True
                    if verbose:
                        print(f"  Cutting arc {u}->{v} from subtour {cyc} (cost {best_cost:.6f})")
                    cut_done = True
                    break

        if not cut_done:
            # If only cycle includes start, cut an arc on that cycle that doesn't enter start
            for cyc in cycles:
                if start in cyc:
                    best_arc = None
                    best_cost = +np.inf
                    for u in cyc:
                        v = nxt[u]
                        if v == start:
                            continue
                        cost_uv = C[u, v]
                        if cost_uv < best_cost:
                            best_cost = cost_uv
                            best_arc = (u, v)
                    if best_arc is not None:
                        u, v = best_arc
                        forbidden[u, v] = True
                        if verbose:
                            print(f"  Cutting arc {u}->{v} from cycle-through-start {cyc} (cost {best_cost:.6f})")
                        cut_done = True
                        break

        if not cut_done:
            # Fallback: cut the globally cheapest selected arc that doesn't enter start
            best_arc = None
            best_cost = +np.inf
            for u, v in nxt.items():
                if v == start:
                    continue
                if C[u, v] < best_cost:
                    best_cost = C[u, v]
                    best_arc = (u, v)
            if best_arc is None:
                raise RuntimeError("Failed to find a cut to enforce path structure.")
            forbidden[best_arc] = True
            if verbose:
                print(f"  Fallback cut: forbidding selected arc {best_arc}->{best_arc[11]} (cost {best_cost:.6f})")

    raise RuntimeError("Maximum iterations exceeded without finding a Hamiltonian path.")

setup_time = np.array([
    [np.inf, 15.05, 12.37, 14.78, 15.23, np.inf],
    [13.70,  np.inf, 15.67, 13.12, 15.46, np.inf],
    [13.51, 13.91,  np.inf, 11.60, 11.86, np.inf],
    [16.49, 16.93, 13.55,  np.inf, 11.88, np.inf],
    [17.00, 16.56, 17.03, 17.28,  np.inf, np.inf],
    [10.58, 16.04, 13.82, 11.54, 14.88, np.inf] 
], dtype=float)

path, cost = solve_atsp_path_assignment(setup_time, start=5, verbose=True)
print("Final path:", path)
print("Total cost:", cost)


Iteration 1: assignment cost = 1000000000061.859985
Assignment pairs: [(0, 2), (1, 3), (2, 1), (3, 4), (4, 5), (5, 0)]
Final path: [5, 0, 2, 1, 3, 4]
Total cost: 61.86
